# Setup

In [1]:
from datetime import datetime, timedelta
from typing import List

from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from termcolor import colored

In [2]:
import functools
import random
from collections import OrderedDict
from typing import Callable, List

import tenacity
from langchain.output_parsers import RegexParser
from langchain.prompts import (
    PromptTemplate,
)
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)
from langchain_openai import ChatOpenAI

In [15]:
import random
import math
import faiss
from langchain_experimental.generative_agents import GenerativeAgent, GenerativeAgentMemory

In [4]:
# Define the model to GPT 3.5 turbo
LLM = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=1500) 

# Create Agent

- [GenerativeAgentMemory](https://python.langchain.com/api_reference/experimental/generative_agents/langchain_experimental.generative_agents.memory.GenerativeAgentMemory.html): **Memory** for the generative agent 
   - `llm`
   - `memory_retriever` = create_new_memory_retriever()
   - `current_plan`
   - `reflection_threshold`
   - `add_memory` add observation/memory

- [GenerativeAgent](https://python.langchain.com/api_reference/experimental/generative_agents.html): Agent as a character with **memory** and innate **characteristics**,  
   - basics like `name`, `age` and `llm`
   - `memory` object that combines relevance, recency, and ‘importance’
   - `summary` and `summary_refresh_seconds` to set how frequently to re-generate the summary
   - `summarize_related_memories`: Summarize memories that are most relevant to an observation
   - `status` fix-objectives / traits of the character you wish not to change
   - `traits` set Permanent traits to ascribe to the character 
   - `generate_dialogue_response`

In [5]:
# Relevance Score function - relevance_score_fn()
def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    return 1.0 - score / math.sqrt(2)

In [6]:
# Memory Retriever function - create_new_memory_retriever()
def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    embeddings_model = OpenAIEmbeddings()  # Define your embedding model
    
    # Initialize the vectorstore as empty
    embedding_size = 1536  # embedding dimension
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),  # empty Memory docstore
        {},  # index-to-document store ID mapping
        relevance_score_fn=relevance_score_fn,
    )

    # Time-weighted scoring mechanism
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore,
        other_score_keys=["importance"],
        k=15  # retrieve up to 15 relevant memories
    )

In [7]:
# Agent Creation function - create_debate_agent()
def create_debate_agent(name, age, traits, status, 
                        #reflection_threshold, 
                        llm):
    
    memory = GenerativeAgentMemory(
        llm=llm,
        memory_retriever=create_new_memory_retriever(),
        verbose=False,
        #reflection_threshold=reflection_threshold,  # adjust as needed for reflection frequency
    )

    agent = GenerativeAgent(
        name=name,
        age=age,
        traits=traits,
        status=status,
        memory_retriever=create_new_memory_retriever(),
        llm=llm,
        memory=memory,
    )
    return agent

In [22]:
# Create three debate agents (MPs) with their respective characteristics

Trott = create_debate_agent(name="Laura Trott", age=38, llm = LLM,
                            traits= "highly disciplined, sharp, and pragmatic. Strategic, focus on “quiet competence” rather than loud rhetoric, detail-oriented and a stickler for facts",
                            status="Conservative MP")

Johnson = create_debate_agent(name="Boris Johnson", age=57, llm = LLM,
                            traits="charismatic, chaotic, opportunistic, larger-than-life personality, thrives on spectacle and Blitz-spirit optimism, mixes humor with charm and a dash of bluster, unpredictable yet captivating, a showman who values headlines over substance",
                            status="Conservative MP")

Farage = create_debate_agent(name="Nigel Farage", age=60, llm = LLM,
                             traits="unapologetically bold, confrontational, divisive, a provocateur, skilled at stirring public opinion with blunt populist rhetoric, political brawle, highly skilled at galvanizing crowds",
                             status="Former UKIP leader, Brexit Party leader, and political commentator")

Sunak = create_debate_agent(name="Rishi Sunak", age=44, llm = LLM,
                            traits="technocratic, astute, polished, financially extremely wealthy, meticulous, highly analytical, known as the Fiscal-Guardian, out of touch with the middle-class",
                            status="Conservative MP, Former Prime Minister")

Starmer = create_debate_agent(name="Sir Keir Starmer", age=61, llm = LLM,
                              traits="methodical, earnest, intense focus on justice and reform, calm demeanor, seeks accountability, values facts over flair, deliver points with precision rather than emotion",
                              status="Leader of the Labour Party")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, sup

In [23]:
# Creat Memory objects for each agent
Trott_memory = Trott.memory
Johnson_memory = Johnson.memory
Farage_memory = Farage.memory   
Sunak_memory = Sunak.memory
Starmer_memory = Starmer.memory

In [24]:
# Base Observations 
Trott_observations = [
    "Trott attended Oxted School, studied history and economics at Oxford University",
    "Trott is preparing for a debate on the economy",
    "Trott advocates for responsible budgeting and cautious government spending",
    "Trott emphasize business growth and pragmatic economic solutions",
    "Trott generally conservative but supports progressive stances on education and family policies",
    "Trott focuses on pragmatic rather than ideological approaches",
]

Johnson_observations = [
    "Johnson attended Eton College, studied Classics Oxford University",
    "Johnson is Pro-Brexit and economically nationalist",
    "Johnson advocates for deregulation, minimal government intervention, and strong support for British businesses",
    "Johnson is a populist, often aligning with traditional conservative values, though flexible when politically advantageous",
    "Johnson is support strong national identity and sovereignty",
]

Farage_observations = [
    "Farage attended Dulwich College but did not attend university",
    "Farage is strongly Eurosceptic, advocates for British sovereignty, deregulation, and cutting ties with EU economic policies",
    "Farage prioritizes domestic industry and independence from European influence",
    "Farage is a Nationalist, anti-globalist, and socially conservative",
    "Farage advocates for strict immigration controls and promotes traditional British values"
]

Sunak_observations = [
    "Sunak studied Philosophy, Politics, and Economics at Oxford University and later earned an MBA from Stanford University",
    "Sunak is fiscal conservative with a focus on budget balancing",
    "Sunak advocates for responsible spending and a cautious approach to government intervention",
    "Sunak prioritizes stability over drastic reforms",
    "Sunak focus on pragmatism over ideology, holds relatively conservative views on social issues, often supporting traditional family values",
]

Starmer_observations = [
    "Starmer attended Reigate Grammar School, studied law at the University of Leeds and completed studies at Oxford University",
    "Starmer focuses on investment in public services, especially the NHS, and progressive taxation",
    "Starmer prioritizes worker rights and social equality, advocating for a balanced but progressive approach",
    "Starmer supports expanded public services, social justice, and inclusivity",
    "Starmer Focuses on social reform and government accountability",
]

In [25]:
# Loop through the observations and add to memory
# Add the observations to the memory using the 'add_memory()' function

for observation in Trott_observations:
    Trott_memory.add_memory(observation)

for observation in Johnson_observations:    
    Johnson_memory.add_memory(observation)

for observation in Farage_observations:
    Farage_memory.add_memory(observation)

for observation in Sunak_observations:
    Sunak_memory.add_memory(observation)

for observation in Starmer_observations:
    Starmer.memory.add_memory(observation)

# Create Simulation

## Framework 1

In [13]:
# Function to run the House of Commons-style debate
def run_house_of_commons_debate(agents: List[GenerativeAgent],
                                initial_observation: str) -> None:
    
    observation = initial_observation
    turns = 0
    max_turns = 20  # Define a max limit to avoid infinite loops
    
    # Start the debate with an initial observation
    print(f"Initial observation: {observation}")
    
    while turns < max_turns:
        break_debate = False
        
        # Each agent independently reacts to the observation
        for agent in agents:
            stay_in_dialogue, response = agent.generate_dialogue_response(observation)
            
            # Record observation in agent's memory
            agent.memory.add_memory(response)
            print(f"{agent.name} says: {response}")
            
            # Update observation for the next agent
            observation = response
            
            if not stay_in_dialogue:
                break_debate = True  # Exit if agent signals end of debate
        
        if break_debate:
            print("Debate ended by an agent.")
            break
        
        turns += 1

In [14]:
# List of agents in the debate
agents = [Trott, Johnson, Farage, Sunak, Starmer]

# Define the initial debate topic
initial_observation = "Donald Trump has been elected as President of the United States for the second time. Should the UK rejoin the European Union?"

# Run the debate
run_house_of_commons_debate(agents, initial_observation)

Initial observation: Donald Trump has been elected as President of the United States for the second time. Should the UK rejoin the European Union?
Laura Trott says: Laura Trott said "As a conservative MP, I believe in carefully considering all options and weighing the potential benefits and drawbacks of rejoining the European Union. It would be important to assess the economic implications, the impact on our sovereignty, and the potential benefits for businesses and citizens. Ultimately, any decision should be made in the best interest of the UK and its people."
Boris Johnson says: Boris Johnson said "Well, Laura, I appreciate your thoughtful approach to such an important decision. As a staunch supporter of Brexit, I believe that leaving the EU was the right choice for our country's sovereignty and future opportunities. However, I understand the need to carefully consider all options and weigh the potential benefits and drawbacks. It's crucial that any decision made is in the best inte

## Framework 2

In [21]:
# List of agents in the debate
agents = [Trott, Johnson, Farage, Sunak, Starmer]

# Define the initial debate topic
initial_observation = "Donald Trump has been elected as President of the United States for the second time. Should the UK rejoin the European Union?"

In [30]:
def personal_idea_generalization(agent, observation):
    # Generate initial thoughts based on traits and status
    return agent.generate_dialogue_response(observation)

def presentation_round(agents, observation):
    random.shuffle(agents)
    for agent in agents:
        idea = personal_idea_generalization(agent, observation)
        print(f"{agent.name} presents: {idea}")

        for other_agent in agents:
            if other_agent != agent:
                other_agent.memory.add_memory(idea)

def discussion_round(agents):
    for _ in range(3):  # Each agent has three speaking opportunities
        for agent in agents:

            # Listen to presentations in memory
            memories = agent.memory.retrieve("presentations")
            
            # Reflect and generate response
            response = agent.generate_dialogue_response(" ".join(memories))
            print(f"{agent.name} speaks: {response}")
            for other_agent in agents:
                if other_agent != agent:
                    other_agent.memory.add_memory(response)


In [31]:
# Run the debate simulation
print("Personal Idea Generalization:")
for agent in agents:
    idea = personal_idea_generalization(agent, initial_observation)
    print(f"{agent.name} initial idea: {idea}")

print("\nPresentation Round:")
presentation_round(agents, initial_observation)

print("\nDiscussion Round:")
discussion_round(agents)

Personal Idea Generalization:
Rishi Sunak initial idea: (True, 'Rishi Sunak said "Given the recent developments in the United States, it\'s clear that the global political landscape is evolving rapidly. As we consider the potential implications of rejoining the European Union, we must approach this decision with caution and prudence. It\'s essential to weigh the benefits and drawbacks carefully, keeping in mind the best interests of our country and its citizens. Let\'s continue to engage in constructive dialogue and work towards finding a solution that serves the needs of the UK as a whole."')
Sir Keir Starmer initial idea: (True, 'Sir Keir Starmer said "While the decision to rejoin the European Union is a complex and multifaceted issue, I believe it is crucial for the UK to carefully consider all the potential benefits and drawbacks. As a leader, my priority is to ensure that any decision made is in the best interest of the country\'s future and the well-being of its citizens. Let\'s 

ValidationError: 1 validation error for Document
page_content
  Input should be a valid string [type=string_type, input_value=(True, 'Sir Keir Starmer ...g a final conclusion."'), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type